In [1]:
import os
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F


In [2]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device=get_default_device()
device

device(type='cuda')

In [3]:
def to_device(data,device):#move tensor to device
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking=True)

In [4]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = datasets.ImageFolder('/kaggle/input/dogs-vs-cats/train', transform=transform)
test_dataset = datasets.ImageFolder('/kaggle/input/dogs-vs-cats/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [5]:
class DeviceDataLoader():
    def __init__(self,dl,device):
        self.dl=dl
        self.device=device

    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)

    def __len__(self):
            return len(self.dl)

In [6]:
test_loader=DeviceDataLoader(test_loader,device)
train_loader=DeviceDataLoader(train_loader,device)

In [7]:

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(128 * 16 * 16, 2) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [8]:
model = SimpleCNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
model =to_device(model,device)

In [10]:
for epoch in range(20):
    total, correct = 0, 0
    for x, y in train_loader:
        out = model(x)
        loss = loss_fn(out, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        correct += (out.argmax(1) == y).sum().item()
        total += y.size(0)

    print(f"Epoch {epoch+1} Accuracy: {100 * correct / total:.2f}%")

Epoch 1 Accuracy: 69.67%
Epoch 2 Accuracy: 77.61%
Epoch 3 Accuracy: 81.22%
Epoch 4 Accuracy: 84.06%
Epoch 5 Accuracy: 85.95%
Epoch 6 Accuracy: 87.25%
Epoch 7 Accuracy: 88.36%
Epoch 8 Accuracy: 89.42%
Epoch 9 Accuracy: 90.25%
Epoch 10 Accuracy: 91.19%
Epoch 11 Accuracy: 91.90%
Epoch 12 Accuracy: 92.81%
Epoch 13 Accuracy: 92.73%
Epoch 14 Accuracy: 93.44%
Epoch 15 Accuracy: 94.14%
Epoch 16 Accuracy: 94.22%
Epoch 17 Accuracy: 94.35%
Epoch 18 Accuracy: 94.81%
Epoch 19 Accuracy: 95.09%
Epoch 20 Accuracy: 95.00%


In [11]:
torch.save(model.state_dict(),'trained_net.pth')

In [12]:
model=SimpleCNN()
model.load_state_dict(torch.load('trained_net.pth'))

/tmp/ipykernel_19/1519630696.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [13]:
model =to_device(model,device)
correct = 0
total = 0

model.eval()  # set network to evaluation mode

with torch.no_grad():  # no gradient computation
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        _, predicted = torch.max(output, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 88.34%
